In [17]:
import os
os.environ['HF_HOME'] = '/expanse/lustre/projects/csd819/a3murali/cache/'

from huggingface_hub import login
login(token="<hf_token>")

import transformers
import torch
import numpy as np
import pandas as pd

print(f"PyTorch Version: {torch.__version__}")
print(f"Transformers Version: {transformers.__version__}")

from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

model_name = "ivnle/llamatales_jr_8b-lay8-hs512-hd8-33M"

# Load config, tokenizer, and model with output_hidden_states enabled
config = AutoConfig.from_pretrained(
    f"{os.environ['HF_HOME']}/models--{model_name.replace('/', '--')}/snapshots/461f50f0024efb46b94dc68cc850d12d75ecb325",
    output_hidden_states=True
)
tokenizer = AutoTokenizer.from_pretrained(
    f"{os.environ['HF_HOME']}/models--{model_name.replace('/', '--')}/snapshots/461f50f0024efb46b94dc68cc850d12d75ecb325",
    config=config
)
model = AutoModelForCausalLM.from_config(config).to("cuda")

# print(model)
# for name, param in model.named_parameters():
#     print(f"{name} -> {param.device}")

# Prompts
prompts = {1: "Once upon a time there was a dragon"}

data = []

for prompt_id, prompt_text in prompts.items():
    for i in range(2):
        # Tokenize input prompt
        inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")

        # Generate output with hidden states

        outputs = model.generate(
            **inputs,
            # attention_mask=inputs.attention_mask,
            max_length=None, max_new_tokens=512, min_length=0, min_new_tokens=None, early_stopping=False, max_time=None, stop_strings=None, do_sample=True, num_beams=1, num_beam_groups=1, penalty_alpha=None, dola_layers=None, use_cache=True, cache_implementation=None, cache_config=None, return_legacy_cache=None, prefill_chunk_size=None, temperature=1.0, top_k=10, top_p=1.0, min_p=None, typical_p=1.0, epsilon_cutoff=0.0, eta_cutoff=0.0, diversity_penalty=0.0, repetition_penalty=1.0, encoder_repetition_penalty=1.0, length_penalty=1.0, no_repeat_ngram_size=0, bad_words_ids=None, force_words_ids=None, renormalize_logits=False, constraints=None, forced_bos_token_id=None, forced_eos_token_id=None, remove_invalid_values=False, exponential_decay_length_penalty=None, suppress_tokens=None, begin_suppress_tokens=None, forced_decoder_ids=None, sequence_bias=None, token_healing=False, guidance_scale=None, low_memory=None, watermarking_config=None, num_return_sequences=1, output_attentions=False, output_hidden_states=True, output_scores=False, output_logits=None, return_dict_in_generate=True, pad_token_id=None, bos_token_id=128000, eos_token_id=128009, encoder_no_repeat_ngram_size=0, decoder_start_token_id=None, is_assistant=False, num_assistant_tokens=20, num_assistant_tokens_schedule='constant', assistant_confidence_threshold=0.4, prompt_lookup_num_tokens=None, max_matching_ngram_size=None, assistant_early_exit=None, assistant_lookbehind=10, target_lookbehind=10, disable_compile=False, generation_kwargs={}, _from_model_config=True, transformers_version="4.52.4")

        # Decode the full generated story
        print(outputs[0])
        generated_story = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
        print(generated_story)

        # Collect hidden states
        raw_hidden_states = outputs.hidden_states  # list of length = num_generated_tokens
        processed_hidden_states = []

        for step_hidden_states in raw_hidden_states:
            # Each step_hidden_states: tuple of layer outputs
            step_layers = [layer.detach().cpu().numpy() for layer in step_hidden_states]
            processed_hidden_states.append(step_layers)

        # Output token prompt id
        num_tokens = len(outputs.sequences[0])
        output_token_prompt_id = np.array([prompt_id] * num_tokens)

        # Save all data
        data.append([prompt_text, generated_story, processed_hidden_states, output_token_prompt_id])

# Save as DataFrame
df = pd.DataFrame(data, columns=["prompt", "story", "hidden_states", "output_token_prompt_id"], dtype="object")

PyTorch Version: 2.7.0+cu126
Transformers Version: 4.52.4


The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


tensor([[128000,  12805,   5304,    264,    892,   1070,    574,    264,  26161,
          32067,  52308,  90655,  50415,  50415,  30891,  86423,  19326,  61392,
         101377,  29526,   6381, 111348,  86789,  34342,  25079,  50415,  36570,
          39053,  23611,  42802,  34342,  13527,  45178,    184, 123549,  79646,
          13527,  26815, 125131,  71672, 106237, 111348, 119176, 101377,  58435,
          36570,  90761,  65298,  12952,  48815,  16593,  19326, 101377,  13527,
          58435, 124600,  28006,  66699,  58435,   2884,  79646,  36570,  49665,
           6381,  25079,  70640, 101352,  38201,  83602,  58435,  52308, 100174,
          32496,  74293,  93518,  78134,  50283,  52561,  42670,  65294,  37945,
          11830, 103005,  60215, 124472,  71764,  49665, 108983, 112761,  12710,
          24680,  69643,  33476,  76976,  49665,  92874, 101377,  51927,  36448,
          71672,  19326,  57709,  83952,  52561,  37548, 110707,  20774,  40306,
         113658, 111735, 118

In [2]:
df

,prompt,story,hidden_states,output_token_prompt_id
0,Once upon a time there was a dragon,Once upon a time there was a dragon.cm)));\n53...,[[[[[-0.00647343 -0.00862156 -0.02739337 ... ...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Once upon a time there was a dragon,Once upon a time there was a dragontributionPl...,[[[[[-0.00647343 -0.00862156 -0.02739337 ... ...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [2]:
# df.to_csv("story_dataset.csv", index = False)